<a href="https://colab.research.google.com/github/lanehale/airline-chatbot/blob/main/pytorch07_exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# See if tensorboard exists, if not, install it
try:
  from torch.utils.tensorboard import SummaryWriter
  print("SummaryWriter already installed.")
except:
  print("Installing SummaryWriter...")
  !pip install -q tensorboard
  from torch.utils.tensorboard import SummaryWriter
  print("Done installing SummaryWriter.")

In [ ]:
"""
Create the going_modular folder and move in its scripts.
"""
import os

# Try to import the going_modular directory, download it from GitHub if it doesn't work
try:
  from going_modular import data_setup, engine
  print("going_modular scripts already downloaded.")
except:
  # Get the going_modular scripts
  print("Downloading going_modular scripts...")
  !git clone https://github.com/lanehale/pytorch-deep-learning
  !mv pytorch-deep-learning/going_modular .
  !rm -rf pytorch-deep-learning
  print("going_modular downloaded.")
  from going_modular import data_setup, engine

print(">!ls going_modular")
!ls going_modular

In [ ]:
import torch
import torchvision

from pathlib import Path
from going_modular import download_data

# Download 10 percent and 20 percent training data
data_10_percent_path = download_data.from_path(from_path="pizza_steak_sushi.zip", image_dir="pizza_steak_sushi")
data_20_percent_path = download_data.from_path(from_path="pizza_steak_sushi_20_percent.zip", image_dir="pizza_steak_sushi_20_percent")

# Set up training directory paths
train_dir_10_percent = data_10_percent_path / "train"
train_dir_20_percent = data_20_percent_path / "train"

# Setup testing directory paths
test_dir_10 = data_10_percent_path / "test"
test_dir_20 = data_20_percent_path / "test"

test_image_path_list_10 = list(Path(test_dir_10).glob("*/*.jpg"))  # this is only used for predictions
test_image_path_list_20 = list(Path(test_dir_20).glob("*/*.jpg"))  # this is only used for predictions

## 1. Pick a larger model from torchvision.models to add to the experiments

In [ ]:
weights_b3 = torchvision.models.EfficientNet_B3_Weights.DEFAULT  # .DEFAULT = best available weights from pretraining on ImageNet
weights_b4 = torchvision.models.EfficientNet_B4_Weights.DEFAULT

"""
model_b3 = torchvision.models.efficientnet_b3(weights=weights_b3)
model_b4 = torchvision.models.efficientnet_b4(weights=weights_b4)

print(f"effnetb3 - {model_b3.classifier}")
print(f"effnetb4 - {model_b4.classifier}")

effnetb3 - Sequential(
  (0): Dropout(p=0.3, inplace=True)
  (1): Linear(in_features=1536, out_features=1000, bias=True)
)
effnetb4 - Sequential(
  (0): Dropout(p=0.4, inplace=True)
  (1): Linear(in_features=1792, out_features=1000, bias=True)
)
"""

# Create models list (need to create a new model for each experiment)
model_parameters = {"effnetb3": {"weights": weights_b3, "in_features": 1536, "dropout": 0.3},
                    "effnetb4": {"weights": weights_b4, "in_features": 1792, "dropout": 0.4},
                    }

# Create datasets list
datasets = [["data_10_percent", train_dir_10_percent, test_dir_10, test_image_path_list_10],
            ["data_20_percent", train_dir_20_percent, test_dir_20, test_image_path_list_20],
            ]

# Create epochs list
num_epochs = [10]

In [ ]:
%%time
"""
Run experiments
"""
from torch import nn
from going_modular import pretrained_writer as pretrained
from going_modular.utils import create_writer, save_model

# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"

BATCH_SIZE = 32
optimizer_type = "Adam"
optimizer_lr = 0.001

# Keep track of experiment numbers
experiment_number = 12

completed_experiments = {}

for model_name in model_parameters:
  weights = model_parameters[model_name]["weights"]
  in_features = model_parameters[model_name]["in_features"]
  dropout = model_parameters[model_name]["dropout"]

  for data in datasets:
    dataset_name = data[0]
    train_dir = data[1]
    test_dir = data[2]
    image_data = data[3]

    for epochs in num_epochs:
      model_to_train = f"{experiment_number}_{model_name}_{dataset_name}_{epochs}_epochs"
      model_to_train_str = model_to_train
      results = "results_" + model_to_train
      predictions = "predictions_" + model_to_train

      if model_name == "effnetb3":
        model_to_train = torchvision.models.efficientnet_b3(weights=weights_b3).to(device)
      elif model_name == "effnetb4":
        model_to_train = torchvision.models.efficientnet_b4(weights=weights_b4).to(device)

      results, predictions = pretrained.run_model_writer(
          model=model_to_train,
          weights=weights,
          train_dir=train_dir,
          test_dir=test_dir,
          batch_size=BATCH_SIZE,
          dropout=dropout,
          in_features=in_features,
          optimizer_type=optimizer_type,
          optimizer_lr=optimizer_lr,
          num_epochs=epochs,
          image_data=image_data,
          device=device,
          writer=create_writer(experiment_name=dataset_name,
                               model_name=model_name,
                               extra=f"{epochs}_epochs"),
          model_name=model_to_train_str
      )

      # Save experiment info for later
      completed_experiments[experiment_number] = [model_to_train_str, model_to_train, results, predictions]

      experiment_number += 1

      # Save the model to file so we can get back the best model
      save_model(model=model_to_train,
                 target_dir="models",
                 model_name=model_to_train_str + ".pth")
      print("-"*50 + "\n")

In [ ]:
!ls models

In [ ]:
from going_modular.utils import compare_results

for experiment in completed_experiments:
  predictions = completed_experiments[experiment][3]
  name = completed_experiments[experiment][0]
  compare_results(predictions, name, 37)

In [ ]:
index = 0
best_model_index = 0
best_model_acc = 0.0

for experiment in completed_experiments:
  results = completed_experiments[experiment][2]
  max_test_acc = max(results['test_acc'])
  index += 1

  if max_test_acc > best_model_acc:
    best_model_acc = max_test_acc
    best_model_index = index

  print(
      f"{completed_experiments[experiment][0] :<46} | "
      f"Max test acc: {max_test_acc:.3f} | "
      f"Min test loss: {min(results['test_loss']):.3f}"
  )

In [ ]:
# Save the models locally to my machine
from google.colab import files

#skip = True
skip = False

if skip:
  for experiment in completed_experiments:
    model_name = completed_experiments[experiment][0]
    model_path = "models/" + model_name + ".pth"  # gets model from here and
    files.download(model_path)                    # saves it to my Downloads folder
  print("Models downloaded.")
else:
  print("Skipping download of models.")

## 2. Introduce data augmentation to the list of experiments

In [ ]:
import torch
import torchvision

from pathlib import Path

weights_b2 = torchvision.models.EfficientNet_B2_Weights.DEFAULT

# Create models list (need to create a new model for each experiment)
models = ["effnetb2_auto",
          "effnetb2_aug_wide_auto",
          "effnetb2_aug_many_auto",
          "effnetb2_norm",
          "effnetb2_aug_wide_norm",
          "effnetb2_aug_many_norm"]

model_parameters = {"weights": weights_b2, "in_features": 1408, "dropout": 0.3}

# Create datasets list
datasets = [["data_20_percent", train_dir_20_percent, test_dir_20, test_image_path_list_20],
            ]

# Create epochs list
num_epochs = [10]

In [ ]:
%%time
"""
Run experiments
"""
from torch import nn
from torchvision import transforms
from going_modular import pretrained_writer as pretrained
from going_modular.utils import create_writer, save_model

# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"

BATCH_SIZE = 32
optimizer_type = "Adam"
optimizer_lr = 0.001
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

# Keep track of experiment numbers
experiment_number = 16

completed_experiments = {}

for model_name in models:
  weights = model_parameters["weights"]
  in_features = model_parameters["in_features"]
  dropout = model_parameters["dropout"]

  for data in datasets:
    dataset_name = data[0]
    train_dir = data[1]
    test_dir = data[2]
    image_data = data[3]

    for epochs in num_epochs:
      model_to_train = f"{experiment_number}_{model_name}_{dataset_name}_{epochs}_epochs"
      model_to_train_str = model_to_train
      results = "results_" + model_to_train
      predictions = "predictions_" + model_to_train

      model_to_train = torchvision.models.efficientnet_b2(weights=weights_b2).to(device)

      if model_name == "effnetb2_auto":
        transform = None  # will default to auto transforms in run_model_writer
      elif model_name == "effnetb2_aug_wide_auto":
        transform = transforms.Compose([
            transforms.TrivialAugmentWide(),
            weights.transforms()
        ])
      elif model_name == "effnetb2_aug_many_auto":
        transform = torchvision.transforms.Compose([
            torchvision.transforms.RandomHorizontalFlip(p=0.5),
            torchvision.transforms.RandomVerticalFlip(p=0.5),
            torchvision.transforms.RandomRotation(degrees=10),
            torchvision.transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
            weights.transforms()
        ])
      elif model_name == "effnetb2_norm":
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            normalize
        ])
      elif model_name == "effnetb2_aug_wide_norm":
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.TrivialAugmentWide(),
            transforms.ToTensor(),
            normalize
        ])
      elif model_name == "effnetb2_aug_many_norm":
        transform = torchvision.transforms.Compose([
            torchvision.transforms.Resize((224, 224)),
            torchvision.transforms.RandomHorizontalFlip(p=0.5),
            torchvision.transforms.RandomVerticalFlip(p=0.5),
            torchvision.transforms.RandomRotation(degrees=10),
            torchvision.transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
            torchvision.transforms.ToTensor(),
            normalize
        ])

      results, predictions = pretrained.run_model_writer(
          model=model_to_train,
          weights=weights,
          train_dir=train_dir,
          test_dir=test_dir,
          batch_size=BATCH_SIZE,
          dropout=dropout,
          in_features=in_features,
          optimizer_type=optimizer_type,
          optimizer_lr=optimizer_lr,
          num_epochs=epochs,
          image_data=image_data,
          device=device,
          writer=create_writer(experiment_name=dataset_name,
                               model_name=model_name,
                               extra=f"{epochs}_epochs"),
          model_name=model_to_train_str,
          transform=transform
      )

      # Save experiment info for later
      completed_experiments[experiment_number] = [model_to_train_str, model_to_train, results, predictions]

      experiment_number += 1

      # Save the model to file so we can get back the best model
      save_model(model=model_to_train,
                 target_dir="models",
                 model_name=model_to_train_str + ".pth")
      print("-"*50 + "\n")

In [ ]:
from going_modular.utils import compare_results

for experiment in completed_experiments:
  predictions = completed_experiments[experiment][3]
  name = completed_experiments[experiment][0]
  compare_results(predictions, name, 51)

In [ ]:
for experiment in completed_experiments:
  results = completed_experiments[experiment][2]
  print(
      f"{completed_experiments[experiment][0] :<51} | "
      f"Max test acc: {max(results['test_acc']):.3f} | "
      f"Min test loss: {min(results['test_loss']):.3f}"
  )

## 3. Scale up the dataset by using the entire Food101 dataset from torchvision.models

In [ ]:
import torch
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms

# Set the root directory where the dataset will be downloaded
root_dir = './data'

# Download all of the Food101 data
datasets.Food101(root=root_dir, download=True)

In [ ]:
!ls data/food-101/images -1 | wc -l
!ls data/food-101/images/waffles -1 | wc -l

In [ ]:
weights_b2 = torchvision.models.EfficientNet_B2_Weights.DEFAULT
auto_transform = weights_b2.transforms()

# Create train and test datasets with transforms
train_dataset_101 = datasets.Food101(root=root_dir, split='train', transform=auto_transform)
test_dataset_101 = datasets.Food101(root=root_dir, split='test', transform=auto_transform)

len(train_dataset_101), len(test_dataset_101)

In [ ]:
"""
If using torchvision.datasets above, the list of all test tensors is already in test_dataset_101.
We could use a different predict_and_store to convert tensors back to images. But usually models
are validated with part of the test data, and predicted on the rest of the test data. Using the
provided Food101 datasets prevents a problem, because the test dataset won't split evenly across
classes.

If using files, this would be all 101,000 images. We'll need to move 25% of each class' images
to test directories to use this method. And in that case we might as well split the test data
into validation and test sets.
"""
# Get a list of all test images for predictions
image_path_list_101 = list(Path("data/food-101/images").glob("*/*.jpg"))

In [ ]:
len(image_path_list_101)

In [ ]:
# Create a function to display each model's file size
from pathlib import Path

def get_model_size(directory_path):
  for root, subdirectories, files in os.walk(directory_path):
    for file in files:
      model_path = os.path.join(root, file)
      model_size = Path(model_path).stat().st_size // (1024 * 1024)
      print(f"Model size of {model_path} trained with 100% data: {model_size} MB")

In [ ]:
# Create a function to copy files to train and test directories
import os
import shutil

def copy_to_split_dirs(source_dir, train_dest_dir, test_dest_dir, n):
  cmd = f"mkdir {train_dest_dir}"
  os.popen(cmd).read()
  cmd = f"mkdir {test_dest_dir}"
  os.popen(cmd).read()

  every_n = 1

  for file_name in os.listdir(source_dir):
    if every_n % n == 0:
      shutil.copy(os.path.join(source_dir, file_name), test_dest_dir)
    else:
      shutil.copy(os.path.join(source_dir, file_name), train_dest_dir)
    every_n += 1

  output = os.popen(f"ls {train_dest_dir} -1 | wc -l").read()
  print(f"{output} images in {train_dest_dir}")
  output = os.popen(f"ls {test_dest_dir} -1 | wc -l").read()
  print(f"{output} images in {test_dest_dir}")

### Try three classes (pizza, steak, sushi) with 100% data

In [ ]:
!mkdir data/food-3
!mkdir data/food-3/images
!mkdir data/food-3/images/train
!mkdir data/food-3/images/test

In [ ]:
n = 1000 / 250  # copy every nth item to test folder

source_dir = 'data/food-101/images/pizza'
test_dest_dir = 'data/food-3/images/test/pizza'
train_dest_dir = 'data/food-3/images/train/pizza'

copy_to_split_dirs(source_dir, train_dest_dir, test_dest_dir, n)

source_dir = 'data/food-101/images/steak'
test_dest_dir = 'data/food-3/images/test/steak'
train_dest_dir = 'data/food-3/images/train/steak'

copy_to_split_dirs(source_dir, train_dest_dir, test_dest_dir, n)

source_dir = 'data/food-101/images/sushi'
test_dest_dir = 'data/food-3/images/test/sushi'
train_dest_dir = 'data/food-3/images/train/sushi'

copy_to_split_dirs(source_dir, train_dest_dir, test_dest_dir, n)

In [ ]:
# Define data directories like usual
train_dir_3 = 'data/food-3/images/train'
test_dir_3 = 'data/food-3/images/test'
test_images_list_3 = list(Path(test_dir_3).glob("*/*.jpg"))

# Create models list
model_parameters = {"effnetb2": {"weights": weights_b2, "in_features": 1408, "dropout": 0.3},
                    }

# Create data paths list
data_paths = [["data_food_3_dir", train_dir_3, test_dir_3, test_images_list_3],
               ]

# Create epochs list
num_epochs = [1]

experiment_number = 28

In [ ]:
%%time
"""
Run experiments
"""
from torch import nn
from torchvision import transforms
from going_modular import pretrained_writer as pretrained
from going_modular.utils import create_writer, save_model

# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"

BATCH_SIZE = 32
optimizer_type = "Adam"
optimizer_lr = 0.001

# Keep track of experiment numbers
#experiment_number = 24

completed_experiments = {}

for model_name in model_parameters:
  weights = model_parameters[model_name]["weights"]
  in_features = model_parameters[model_name]["in_features"]
  dropout = model_parameters[model_name]["dropout"]

  for data in data_paths:
    dataset_name = data[0]
    train_dir = data[1]
    test_dir = data[2]
    image_data = data[3]

    for epochs in num_epochs:
      model_to_train = f"{experiment_number}_{model_name}_{dataset_name}_{epochs}_epochs"
      model_to_train_str = model_to_train
      results = "results_" + model_to_train
      predictions = "predictions_" + model_to_train

      model_to_train = torchvision.models.efficientnet_b2(weights=weights_b2).to(device)

      results, predictions = pretrained.run_model_writer(
          model=model_to_train,
          weights=weights,
          train_dir=train_dir,
          test_dir=test_dir,
          batch_size=BATCH_SIZE,
          dropout=dropout,
          in_features=in_features,
          optimizer_type=optimizer_type,
          optimizer_lr=optimizer_lr,
          num_epochs=epochs,
          image_data=image_data,
          device=device,
          writer=create_writer(experiment_name=dataset_name,
                               model_name=model_name,
                               extra=f"{epochs}_epochs"),
          model_name=model_to_train_str
      )

      # Save experiment info for later
      completed_experiments[experiment_number] = [model_to_train_str, model_to_train, results, predictions]

      experiment_number += 1

      # Save the model to file so we can get back the best model
      save_model(model=model_to_train,
                 target_dir="models",
                 model_name=model_to_train_str + ".pth")
      print("-"*50 + "\n")

### end of experiments ###

print()

# Display results
from going_modular.utils import compare_results

for experiment in completed_experiments:
  predictions = completed_experiments[experiment][3]
  name = completed_experiments[experiment][0]
  compare_results(predictions, name, 40)

print()

# Display the model file size
from pathlib import Path

directory_path = "models"
get_model_size(directory_path)

### Run the full Food101 dataset

In [ ]:
import os
import shutil

def move_to_split_dirs(source_dir, train_dest_dir, test_dest_dir, total_items, num_to_move, split=False):
  cmd = f"mkdir {train_dest_dir}"
  os.popen(cmd).read()
  cmd = f"mkdir {test_dest_dir}"
  os.popen(cmd).read()

  n = total_items // num_to_move  # move every nth item to test_dest folder
  every_n = 1
  count = 0

  for file_name in os.listdir(source_dir):
    if every_n % n == 0 and count < num_to_move:
      shutil.move(os.path.join(source_dir, file_name), test_dest_dir)
      count += 1
    elif split:
      shutil.move(os.path.join(source_dir, file_name), train_dest_dir)
    every_n += 1

  output = os.popen(f"ls {train_dest_dir} -1 | wc -l").read()
  print(f"{output} images in {train_dest_dir}")
  output = os.popen(f"ls {test_dest_dir} -1 | wc -l").read()
  print(f"{output} images in {test_dest_dir}")

In [ ]:
!mkdir data/food-101/images/train
!mkdir data/food-101/images/test
!mkdir data/food-101/images/val

### Each class 700 train, 175 validation, 125 test images

In [ ]:
# Separate data into train and test directories
for class_name in train_dataset_101.classes:

  source_dir = f'data/food-101/images/{class_name}'
  test_dest_dir = f'data/food-101/images/val/{class_name}'
  train_dest_dir = f'data/food-101/images/train/{class_name}'

  # Move 300 to validation
  move_to_split_dirs(source_dir, train_dest_dir, test_dest_dir, 1000, 300, True)

In [ ]:
# Separate test data into validation and test directories
for class_name in train_dataset_101.classes:

  source_dir = f'data/food-101/images/val/{class_name}'
  test_dest_dir = f'data/food-101/images/test/{class_name}'
  train_dest_dir = f'data/food-101/images/val/{class_name}'

  # Move 125 to test
  move_to_split_dirs(source_dir, train_dest_dir, test_dest_dir, 300, 125)

In [ ]:
train_data_101 = "data/food-101/images/train"
val_data_101 = "data/food-101/images/val"
test_data_101 = "data/food-101/images/test"
test_data_path_list_101 = list(Path(test_data_101).glob("*/*.jpg"))

!ls {train_data_101} -1 | wc -l
!ls {val_data_101} -1 | wc -l
!ls {test_data_101} -1 | wc -l
len(test_data_path_list_101)

In [ ]:
# Create models list
model_parameters = {"effnetb2": {"weights": weights_b2, "in_features": 1408, "dropout": 0.3},
                    }

# Create data paths list
data_paths = [["data_food_101", train_data_101, val_data_101, test_data_path_list_101],
              ]

# Create epochs list
num_epochs = [5]

experiment_number = 24

In [ ]:
%%time
"""
Run experiments
"""
from torch import nn
from torchvision import transforms
from going_modular import pretrained_writer as pretrained
from going_modular.utils import create_writer, save_model

# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"

BATCH_SIZE = 32
optimizer_type = "Adam"
optimizer_lr = 0.001

# Keep track of experiment numbers
#experiment_number = 24

completed_experiments = {}

for model_name in model_parameters:
  weights = model_parameters[model_name]["weights"]
  in_features = model_parameters[model_name]["in_features"]
  dropout = model_parameters[model_name]["dropout"]

  for data in data_paths:
    dataset_name = data[0]
    train_dir = data[1]
    test_dir = data[2]
    image_data = data[3]

    for epochs in num_epochs:
      model_to_train = f"{experiment_number}_{model_name}_{dataset_name}_{epochs}_epochs"
      model_to_train_str = model_to_train
      results = "results_" + model_to_train
      predictions = "predictions_" + model_to_train

      model_to_train = torchvision.models.efficientnet_b2(weights=weights_b2).to(device)

      results, predictions = pretrained.run_model_writer(
          model=model_to_train,
          weights=weights,
          train_dir=train_dir,
          test_dir=test_dir,
          batch_size=BATCH_SIZE,
          dropout=dropout,
          in_features=in_features,
          optimizer_type=optimizer_type,
          optimizer_lr=optimizer_lr,
          num_epochs=epochs,
          image_data=image_data,
          device=device,
          writer=create_writer(experiment_name=dataset_name,
                               model_name=model_name,
                               extra=f"{epochs}_epochs"),
          model_name=model_to_train_str
      )

      # Save experiment info for later
      completed_experiments[experiment_number] = [model_to_train_str, model_to_train, results, predictions]

      experiment_number += 1

      # Save the model to file so we can get back the best model
      save_model(model=model_to_train,
                 target_dir="models",
                 model_name=model_to_train_str + ".pth")
      print("-"*50 + "\n")

### end of experiments ###

print()

# Display results
from going_modular.utils import compare_results

for experiment in completed_experiments:
  predictions = completed_experiments[experiment][3]
  name = completed_experiments[experiment][0]
  compare_results(predictions, name, 40)

print()

# Display the model file size
from pathlib import Path

directory_path = "models"
get_model_size(directory_path)

### Each class 600 train, 200 validation, 200 test images

In [ ]:
# Separate data into train and test directories
for class_name in train_dataset_101.classes:

  source_dir = f'data/food-101/images/{class_name}'
  test_dest_dir = f'data/food-101/images/val/{class_name}'
  train_dest_dir = f'data/food-101/images/train/{class_name}'

  # Move 400 to validation
  move_to_split_dirs(source_dir, train_dest_dir, test_dest_dir, 1000, 400, True)

In [ ]:
# Separate test data into validation and test directories
for class_name in train_dataset_101.classes:

  source_dir = f'data/food-101/images/val/{class_name}'
  test_dest_dir = f'data/food-101/images/test/{class_name}'
  train_dest_dir = f'data/food-101/images/val/{class_name}'

  # Move 200 to test
  move_to_split_dirs(source_dir, train_dest_dir, test_dest_dir, 400, 200)

In [ ]:
train_data_101 = "data/food-101/images/train"
val_data_101 = "data/food-101/images/val"
test_data_101 = "data/food-101/images/test"
test_data_path_list_101 = list(Path(test_data_101).glob("*/*.jpg"))

!ls {train_data_101} -1 | wc -l
!ls {val_data_101} -1 | wc -l
!ls {test_data_101} -1 | wc -l
len(test_data_path_list_101)

In [ ]:
# Create models list
model_parameters = {"effnetb2": {"weights": weights_b2, "in_features": 1408, "dropout": 0.3},
                    }

# Create data paths list
data_paths = [["data_food_101", train_data_101, val_data_101, test_data_path_list_101],
              ]

# Create epochs list
num_epochs = [5]

experiment_number = 25

In [ ]:
%%time
"""
Run experiments
"""
from torch import nn
from torchvision import transforms
from going_modular import pretrained_writer as pretrained
from going_modular.utils import create_writer, save_model

# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"

BATCH_SIZE = 32
optimizer_type = "Adam"
optimizer_lr = 0.001

# Keep track of experiment numbers
#experiment_number = 24

completed_experiments = {}

for model_name in model_parameters:
  weights = model_parameters[model_name]["weights"]
  in_features = model_parameters[model_name]["in_features"]
  dropout = model_parameters[model_name]["dropout"]

  for data in data_paths:
    dataset_name = data[0]
    train_dir = data[1]
    test_dir = data[2]
    image_data = data[3]

    for epochs in num_epochs:
      model_to_train = f"{experiment_number}_{model_name}_{dataset_name}_{epochs}_epochs"
      model_to_train_str = model_to_train
      results = "results_" + model_to_train
      predictions = "predictions_" + model_to_train

      model_to_train = torchvision.models.efficientnet_b2(weights=weights_b2).to(device)

      results, predictions = pretrained.run_model_writer(
          model=model_to_train,
          weights=weights,
          train_dir=train_dir,
          test_dir=test_dir,
          batch_size=BATCH_SIZE,
          dropout=dropout,
          in_features=in_features,
          optimizer_type=optimizer_type,
          optimizer_lr=optimizer_lr,
          num_epochs=epochs,
          image_data=image_data,
          device=device,
          writer=create_writer(experiment_name=dataset_name,
                               model_name=model_name,
                               extra=f"{epochs}_epochs"),
          model_name=model_to_train_str
      )

      # Save experiment info for later
      completed_experiments[experiment_number] = [model_to_train_str, model_to_train, results, predictions]

      experiment_number += 1

      # Save the model to file so we can get back the best model
      save_model(model=model_to_train,
                 target_dir="models",
                 model_name=model_to_train_str + ".pth")
      print("-"*50 + "\n")

In [ ]:
from going_modular.utils import compare_results

for experiment in completed_experiments:
  predictions = completed_experiments[experiment][3]
  name = completed_experiments[experiment][0]
  compare_results(predictions, name, 40)

In [ ]:
# Check the model file size
from pathlib import Path

directory_path = "models"
get_model_size(directory_path)

### Each class 750 train, 200 validation, 50 test images

In [ ]:
# Separate data into train and test directories
for class_name in train_dataset_101.classes:

  source_dir = f'data/food-101/images/{class_name}'
  test_dest_dir = f'data/food-101/images/val/{class_name}'
  train_dest_dir = f'data/food-101/images/train/{class_name}'

  # Move 250 to validation
  move_to_split_dirs(source_dir, train_dest_dir, test_dest_dir, 1000, 250, True)

In [ ]:
# Separate test data into validation and test directories
for class_name in train_dataset_101.classes:

  source_dir = f'data/food-101/images/val/{class_name}'
  test_dest_dir = f'data/food-101/images/test/{class_name}'
  train_dest_dir = f'data/food-101/images/val/{class_name}'

  # Move 50 to test
  move_to_split_dirs(source_dir, train_dest_dir, test_dest_dir, 250, 50)

In [ ]:
train_data_101 = "data/food-101/images/train"
val_data_101 = "data/food-101/images/val"
test_data_101 = "data/food-101/images/test"
test_data_path_list_101 = list(Path(test_data_101).glob("*/*.jpg"))

!ls {train_data_101} -1 | wc -l
!ls {val_data_101} -1 | wc -l
!ls {test_data_101} -1 | wc -l
len(test_data_path_list_101)

In [ ]:
# Create models list
model_parameters = {"effnetb2": {"weights": weights_b2, "in_features": 1408, "dropout": 0.3},
                    }

# Create data paths list
data_paths = [["data_food_101", train_data_101, val_data_101, test_data_path_list_101],
              ]

# Create epochs list
num_epochs = [5]

experiment_number = 26

In [ ]:
%%time
"""
Run experiments
"""
from torch import nn
from torchvision import transforms
from going_modular import pretrained_writer as pretrained
from going_modular.utils import create_writer, save_model

# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"

BATCH_SIZE = 32
optimizer_type = "Adam"
optimizer_lr = 0.001

# Keep track of experiment numbers
#experiment_number = 24

completed_experiments = {}

for model_name in model_parameters:
  weights = model_parameters[model_name]["weights"]
  in_features = model_parameters[model_name]["in_features"]
  dropout = model_parameters[model_name]["dropout"]

  for data in data_paths:
    dataset_name = data[0]
    train_dir = data[1]
    test_dir = data[2]
    image_data = data[3]

    for epochs in num_epochs:
      model_to_train = f"{experiment_number}_{model_name}_{dataset_name}_{epochs}_epochs"
      model_to_train_str = model_to_train
      results = "results_" + model_to_train
      predictions = "predictions_" + model_to_train

      model_to_train = torchvision.models.efficientnet_b2(weights=weights_b2).to(device)

      results, predictions = pretrained.run_model_writer(
          model=model_to_train,
          weights=weights,
          train_dir=train_dir,
          test_dir=test_dir,
          batch_size=BATCH_SIZE,
          dropout=dropout,
          in_features=in_features,
          optimizer_type=optimizer_type,
          optimizer_lr=optimizer_lr,
          num_epochs=epochs,
          image_data=image_data,
          device=device,
          writer=create_writer(experiment_name=dataset_name,
                               model_name=model_name,
                               extra=f"{epochs}_epochs"),
          model_name=model_to_train_str
      )

      # Save experiment info for later
      completed_experiments[experiment_number] = [model_to_train_str, model_to_train, results, predictions]

      experiment_number += 1

      # Save the model to file so we can get back the best model
      save_model(model=model_to_train,
                 target_dir="models",
                 model_name=model_to_train_str + ".pth")
      print("-"*50 + "\n")

### end of experiments ###

print()

# Display results
from going_modular.utils import compare_results

for experiment in completed_experiments:
  predictions = completed_experiments[experiment][3]
  name = completed_experiments[experiment][0]
  compare_results(predictions, name, 40)

print()

# Display the model file size
from pathlib import Path

directory_path = "models"
get_model_size(directory_path)